In [1]:
import os
from nltk.corpus import stopwords
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer ,LancasterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob , Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score ,f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('IMDB Dataset.csv')
print(df.shape)
df.head(5)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
def process(review):
    # review without HTML tags
    review = BeautifulSoup(review).get_text()
    # review without punctuation and numbers
    review = re.sub("[^a-zA-Z]",' ',review)
    # converting into lowercase and splitting to eliminate stopwords
    review = review.lower()
    review = review.split()
    # review without stopwords
    swords = set(stopwords.words("english")) 
    review = [w for w in review if w not in swords]               
    # splitted paragraph space return
    return(" ".join(review))

In [4]:
def lemma(data):
    lemmatizer=WordNetLemmatizer()
    out_data=""
    for words in data:
        out_data+= lemmatizer.lemmatize(words)
    return out_data

df['review']=df['review'].apply(lambda z: lemma(z))

In [5]:
train_data = []
for i in range(len(df["review"])):        
    train_data.append(process(df["review"][i]))

In [6]:
independent = train_data
dependent = np.array(df["sentiment"])
dependent[dependent == 'positive'] = 1
dependent[dependent == 'negative'] = 0

train_x, test_x, y_train, y_test = train_test_split(independent,dependent, stratify = dependent ,test_size = 0.2, shuffle = True , random_state = 25)

In [7]:
vect = TfidfVectorizer(max_features = 15000 , ngram_range=(1,3))
train_x = vect.fit_transform(train_x)
test_x = vect.transform(test_x)
print( train_x.shape , test_x.shape)

(40000, 15000) (10000, 15000)


In [8]:
X_train = train_x.toarray()
X_test = test_x.toarray()

In [9]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [10]:
clf = RandomForestClassifier(n_estimators = 3 , random_state=44 , n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [11]:
print('train score: ' ,clf.score(X_train, y_train))
print('test score: ' ,clf.score(X_test, y_test))
print('accuracy score: ' ,accuracy_score(y_pred, y_test))
print('F1 Score : ',f1_score(y_test, y_pred, average='weighted'))
print('confusion matrix: \n' , confusion_matrix(y_test,y_pred))

train score:  0.953725
test score:  0.7349
accuracy score:  0.7349
F1 Score :  0.7348974523645172
confusion matrix: 
 [[3659 1341]
 [1310 3690]]


In [12]:
parameters = {
    'n_estimators' : [3 , 5 , 10 , 15] ,
    'max_depth' : [5,7,10],
    'min_samples_split' :[25,50,100], 
    'n_jobs' : [-1]
  }

In [13]:
clf2 = RandomForestClassifier()

In [15]:
grid = GridSearchCV(clf2, param_grid=parameters)
grid.fit(X_train,y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 7, 10],
                         'min_samples_split': [25, 50, 100],
                         'n_estimators': [3, 5, 10, 15], 'n_jobs': [-1]})

In [16]:
print('Best estimator : ' , grid.best_estimator_)

print('Best params:' , grid.best_params_)

Best estimator :  RandomForestClassifier(max_depth=10, min_samples_split=25, n_estimators=15,
                       n_jobs=-1)
Best params: {'max_depth': 10, 'min_samples_split': 25, 'n_estimators': 15, 'n_jobs': -1}


In [17]:
ygrid = grid.predict(X_test)
print('grid train score: ' , grid.score(X_train,y_train))
print('grid test score: ' , grid.score(X_test,y_test))
print('grid accuracy score: ' , accuracy_score(ygrid,y_test))
print('grid f1 score: ' , f1_score(ygrid,y_test))

grid train score:  0.787325
grid test score:  0.7623
grid accuracy score:  0.7623
grid f1 score:  0.7793967517401393


In [19]:
import pickle as pk
pk.dump(grid , open('saved file2.sav','wb'))
